# Введение

В этой тетрадке я реализую свою русскоязычную QA систему.
Так как в задании сказано, что  ответ уже есть в заданном вопросе, 
я буду использовать подход поиска start/end токенов ответа в вопросе, 
и буду подавать моделе сначала контекст, потом вопрос, не разделяя их.

В качестве базовой модели я возьму энкодер *sergeyzh/LaBSE-ru-turbo*, который обладает средним размером (формата bert-base) и прекрасным качеством (топ 2 энкодечки).

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


splits = {'train': 'sberquad/train-00000-of-00001.parquet', 'validation': 'sberquad/validation-00000-of-00001.parquet', 'test': 'sberquad/test-00000-of-00001.parquet'}
train = pd.read_parquet("hf://datasets/kuznetsoffandrey/sberquad/" + splits["train"])
val = pd.read_parquet("hf://datasets/kuznetsoffandrey/sberquad/" + splits["validation"])

# Подготовим данные

In [2]:
# немного подправим. Понимаю, что выглядит не очень, но я потратил на это достаточно много времени
train.iloc[43573].answers['text'][0] = '573,000'
train.iloc[223].answers['text'][0] = 'о' + train.iloc[223].answers['text'][0]
train.iloc[308].answers['text'][0] = train.iloc[308].answers['text'][0].lower()
train.iloc[415].answers['text'][0] = 'к' + train.iloc[415].answers['text'][0]
train.iloc[587].answers['text'][0] = '1' + train.iloc[587].answers['text'][0]
train.iloc[848].answers['text'][0] = 'магнитофонов'
train.iloc[1300].answers['text'][0] = 'н' + train.iloc[1300].answers['text'][0]
train.iloc[1360].answers['text'][0] = 'Ратхиса'
train.iloc[1626].answers['text'][0] = 'Й. Яблонскиса'
train.iloc[1874].answers['text'][0] = '1781 года'
train.iloc[2850].answers['text'][0] = train.iloc[2850].answers['text'][0][:-1] + 'я'
train.iloc[2974].answers['text'][0] = train.iloc[2974].answers['text'][0][:-1] + 'а'
train.iloc[3077].answers['text'][0] = train.iloc[3077].answers['text'][0][:-2]
train.iloc[3629].answers['text'][0] = train.iloc[3629].answers['text'][0] + 'й'
train.iloc[3731].answers['text'][0] = train.iloc[3731].answers['text'][0] + 'м'
train.iloc[3750].answers['text'][0] = train.iloc[3750].answers['text'][0] + 'а'
train.iloc[4574].answers['text'][0] = train.iloc[4574].answers['text'][0] + 'а'
train.iloc[4767].answers['text'][0] = 'Полиморфизм'
train.iloc[4779].answers['text'][0] = 'Полиморфизм'
train.iloc[5341].answers['text'][0] = train.iloc[5341].answers['text'][0] + 'а'
train.iloc[6263].answers['text'][0] = train.iloc[6263].answers['text'][0] + 'о'
train.iloc[6409].answers['text'][0] = train.iloc[6409].answers['text'][0] + 'а'
train.iloc[6505].answers['text'][0] = train.iloc[6505].answers['text'][0] + 'а'
train.iloc[6535].answers['text'][0] = 'э' + train.iloc[6535].answers['text'][0]
train.iloc[6610].answers['text'][0] = 'к' + train.iloc[6610].answers['text'][0]
train.iloc[6711].answers['text'][0] = train.iloc[6711].answers['text'][0] + 'а'
train.iloc[6897].answers['text'][0] = train.iloc[6897].answers['text'][0] + 'й'
train.iloc[7812].answers['text'][0] = train.iloc[7812].answers['text'][0] + 'одах'
train.iloc[8196].answers['text'][0] = train.iloc[8196].answers['text'][0] + 'и'
train.iloc[8266].answers['text'][0] = train.iloc[8266].answers['text'][0] + 'у'
train.iloc[8526].answers['text'][0] = train.iloc[8526].answers['text'][0] + 'ом'
train.iloc[8666].answers['text'][0] = train.iloc[8666].answers['text'][0] + 'а'
train.iloc[9332].answers['text'][0] = train.iloc[9332].answers['text'][0] + 'а'
train.iloc[9333].answers['text'][0] = train.iloc[9333].answers['text'][0] + 'а'
train.iloc[9459].answers['text'][0] = 'о' + train.iloc[9459].answers['text'][0]
train.iloc[10150].answers['text'][0] = 'р' + train.iloc[10150].answers['text'][0]
train.iloc[10408].answers['text'][0] = train.iloc[10408].answers['text'][0] + 'ом'
train.iloc[10845].answers['text'][0] = 'г' + train.iloc[10845].answers['text'][0]
train.iloc[11082].answers['text'][0] = train.iloc[11082].answers['text'][0] + 'т'
train.iloc[14821].answers['text'][0] = train.iloc[14821].answers['text'][0] + 'в'
train.iloc[19574].answers['text'][0] = train.iloc[19574].answers['text'][0] + 'а'
train.iloc[26674].answers['text'][0] = train.iloc[26674].answers['text'][0][:-1] + 'ом'
train.iloc[30344].answers['text'][0] = train.iloc[30344].answers['text'][0] + 'а'
train.iloc[33116].answers['text'][0] = train.iloc[33116].answers['text'][0] + 'а'
train.iloc[37272].answers['text'][0] = train.iloc[37272].answers['text'][0][:-1] + 'а'
train.iloc[39608].answers['text'][0] = 'десять'
train.iloc[43097].answers['text'][0] = train.iloc[43097].answers['text'][0] + 'ь'

In [3]:
import string

# иногда ответ находится в другом регистре или с другими знаками препинания, исправим это, 
# приведя ответы к такому же виду, в котором они содержатся в контексе
for i in range(train.shape[0]):
    train.iloc[i].answers['text'][0] = train.iloc[i].answers['text'][0].strip(string.punctuation).strip()
    ans = train.iloc[i].answers['text'][0]
    text = train.iloc[i].context
    if ans not in text:
        ind = text.lower().find(ans.lower())
        if ind == -1:
            print(i)
            print(ans, '\n', text)
        else:
            train.iloc[i].answers['text'][0] = text[ind: ind + len(ans)].strip()

In [4]:
# то же самое с валидационной выборкой
for i in range(val.shape[0]):
    val.iloc[i].answers['text'][0] = val.iloc[i].answers['text'][0].strip(string.punctuation).strip()
    ans = val.iloc[i].answers['text'][0]
    text = val.iloc[i].context
    if ans not in text:
        ind = text.lower().find(ans.lower())
        if ind == -1:
            print(i)
            print(ans, '\n', text)
        else:
            val.iloc[i].answers['text'][0] = text[ind: ind + len(ans)].strip()

In [5]:
train.shape, val.shape

((45328, 5), (5036, 5))

In [6]:
train.tail()

,id,title,context,question,answers
45323,6601,SberChallenge,"Познакомившись с двигателем Ленуара, осенью 18...",Когда подали заявку на патент на двигатель с ж...,"{'text': ['в январе 1861 года'], 'answer_start..."
45324,84192,SberChallenge,"Познакомившись с двигателем Ленуара, осенью 18...",Что создал после отклонения заявки Николаус Ау...,{'text': ['двухтактный атмосферный двигатель в...
45325,38284,SberChallenge,Главную роль в истории с письменным обязательс...,Что было целью разыгранного Веберами спектакля?,"{'text': ['сближение Моцарта с Констанцией'], ..."
45326,73427,SberChallenge,Главную роль в истории с письменным обязательс...,Что не мог подписать Моцарт из-за сильно разви...,"{'text': ['заявление'], 'answer_start': [376]}"
45327,10726,SberChallenge,Главную роль в истории с письменным обязательс...,Какое имя у опекуна Констанции и её сестёр?,"{'text': ['Иоганн Торварт'], 'answer_start': [..."


In [7]:
train.context.str.len().describe()

count    45328.000000
mean       753.740955
std        259.313862
min        279.000000
25%        577.000000
50%        682.000000
75%        859.000000
max       7231.000000
Name: context, dtype: float64

In [8]:
train.question.str.len().describe()

count    45328.000000
mean        64.393818
std         32.342134
min          8.000000
25%         44.000000
50%         58.000000
75%         77.000000
max        670.000000
Name: question, dtype: float64

In [9]:
val.context.str.len().describe()

count    5036.000000
mean      754.883836
std       260.708452
min       279.000000
25%       578.000000
50%       680.000000
75%       857.000000
max      3489.000000
Name: context, dtype: float64

In [10]:
val.question.str.len().describe()

count    5036.000000
mean       64.172160
std        31.028467
min        10.000000
25%        43.000000
50%        58.000000
75%        77.000000
max       471.000000
Name: question, dtype: float64

Большинство должно влезть в контекстное окно

In [11]:
sum([i['text'][0] == '' for i in val.answers])

0

In [12]:
sum([i['text'][0] == '' for i in train.answers])

0

# Подготовим train выборку для модели

In [13]:
import torch
from transformers import AutoTokenizer

weight_path = "sergeyzh/LaBSE-ru-turbo"
tokenizer = AutoTokenizer.from_pretrained(weight_path, use_fast=True)

In [14]:
context_question = (train["context"] + ' ' + train["question"]).to_list()
answers = train["answers"].to_list()
MAX_LEN = 512

train_inputs = tokenizer(
    context_question,
    max_length=MAX_LEN,
    truncation=True,
    padding='max_length'
)

answers_tokens = tokenizer([i['text'][0] for i in answers], add_special_tokens=False).input_ids

Пробовал разные стратегии, но остановился на простом поиске токенов ответа в токенах контекста

In [15]:
start_positions = []
end_positions = []
errors = 0
drop = []
for idx, input_ids in enumerate(train_inputs.input_ids):

    answer_tokens = answers_tokens[idx]

    index = -1
    for i in range(len(input_ids) - len(answer_tokens) + 1):
        if input_ids[i:i+len(answer_tokens)] == answer_tokens:
            index = i
            break

    if index != -1:
        start_positions.append(index)
        end_positions.append(index + len(answer_tokens))
    else:
        errors += 1
        drop.append(idx)
        
print(f'В тренировочную выборку не попало {errors} обьектов, в которых допущены ошибки')

В тренировочную выборку не попало 148 обьектов, в которых допущены ошибки


In [16]:
new_train_input_ids = np.delete(train_inputs.input_ids, drop, axis=0).tolist()
new_train_attention_mask = np.delete(train_inputs.attention_mask, drop, axis=0).tolist()
new_answers = np.delete(answers, drop, axis=0).tolist()

In [17]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 40

train_dataset = TensorDataset(torch.tensor(new_train_input_ids), 
                              torch.tensor(new_train_attention_mask), 
                              torch.tensor(start_positions), 
                              torch.tensor(end_positions))
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler = train_sampler, batch_size = batch_size)

# Подготовим validation выборку для модели

In [18]:
context_question = (val["context"] + ' ' + val["question"]).to_list()
answers = val["answers"].to_list()

val_inputs = tokenizer(
    context_question,
    max_length=MAX_LEN,
    truncation=True,
    padding='max_length'
)

answers_tokens = tokenizer([i['text'][0] for i in answers], add_special_tokens=False).input_ids

In [19]:
start_positions = []
end_positions = []
errors = 0
drop = []

for idx, input_ids in enumerate(val_inputs.input_ids):

    answer_tokens = answers_tokens[idx]

    index = -1
    for i in range(len(input_ids) - len(answer_tokens) + 1):
        if input_ids[i:i+len(answer_tokens)] == answer_tokens:
            index = i
            break

    if index != -1:
        start_positions.append(index)
        end_positions.append(index + len(answer_tokens))
    else:
        errors += 1
        drop.append(idx)

print(f'В валидационную выборку не попало {errors} обьектов, в которых допущены ошибки')

В валидационную выборку не попало 13 обьектов, в которых допущены ошибки


In [20]:
new_val_input_ids = np.delete(val_inputs.input_ids, drop, axis=0).tolist()
new_val_attention_mask = np.delete(val_inputs.attention_mask, drop, axis=0).tolist()
new_answers = np.delete(answers, drop, axis=0).tolist()

In [21]:
val_dataset = TensorDataset(torch.tensor(new_val_input_ids), 
                              torch.tensor(new_val_attention_mask), 
                              torch.tensor(start_positions), 
                              torch.tensor(end_positions))
val_sampler = SequentialSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler = val_sampler, batch_size = batch_size)

# Обучим модель

In [22]:
import torch
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(weight_path)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device);

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at sergeyzh/LaBSE-ru-turbo and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
from tqdm import tqdm, trange
from torch.optim.lr_scheduler import StepLR

def evaluate():
    model.eval()
    preds, ans = [], []
    
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc='Val'):
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            start_positions, end_positions = batch[2:]
        
            result = model(input_ids = input_ids, attention_mask = attention_mask,return_dict=True)
        
            start_preds = np.argmax(result.start_logits.detach().cpu(), axis=1)
            end_preds = np.argmax(result.end_logits.detach().cpu(), axis=1)
            input_ids_cpu = input_ids.detach().cpu().tolist()
            preds.extend([input_ids_cpu[i][start_preds[i]:end_preds[i]] if start_preds[i] < end_preds[i] else [] 
                          for i in range(start_preds.shape[0])])
            ans.extend([input_ids_cpu[i][start_positions[i]:end_positions[i]] if start_positions[i] < end_positions[i] else [] 
                        for i in range(start_positions.shape[0])])
    
    exact_match = np.mean([ans[i] == preds[i] for i in range(len(ans))])
    
    common_tokens = [set(preds[i]) & set(ans[i]) for i in range(len(ans))]
    prec = np.mean([len(common_tokens[i]) / len(preds[i]) if len(preds[i]) != 0 else preds[i] == ans[i] for i in range(len(ans))])
    rec = np.mean([len(common_tokens[i]) / len(ans[i]) if len(ans[i]) != 0 else preds[i] == ans[i] for i in range(len(ans))])
    f1 = 2 * (prec * rec) / (prec + rec)
    
    print(f'EM: {exact_match} | F1: {f1}')
    return exact_match, f1


def train(epochs, best_f1, best_exact_match, scheduler=None):
    for epoch in range(epochs):
        training_loss = 0
        model.train()
        t = tqdm(train_dataloader)
        for i, batch in enumerate(t):
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, start_positions, end_positions = batch
            
            result = model(input_ids = input_ids, 
                            attention_mask = attention_mask,
                            start_positions = start_positions,
                            end_positions = end_positions,
                            return_dict = True)
    
            optimizer.zero_grad(set_to_none=True)
            loss = result.loss
            training_loss += loss.item()      
            loss.backward()
            optimizer.step()
    
            avg_train_loss = training_loss / (i + 1) 
            t.set_description(f"Training loss for batch: {loss.item()}")
            # if (i + 1) % 200 == 0:
            #     print(f'Average training loss: {avg_train_loss}')  
    
        print(f'EPOCH {epoch + 1} AVARAGE TRAINING LOSS: {avg_train_loss}')
        
        print('\nVALIDATION:')
        exact_match, f1 = evaluate()
        if f1 > best_f1 and exact_match > best_exact_match:
            best_f1 = f1
            best_exact_match = exact_match
            torch.save(model.state_dict(), 'best_base_model.pt')
            print('Model saved!')
        print()

        if scheduler:
            scheduler.step()
        torch.cuda.empty_cache()
    return best_f1, best_exact_match

In [24]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3) 
epochs = 8
best_f1 = 0
best_exact_match = 0

best_f1, best_exact_match = train(epochs, best_f1, best_exact_match)

Training loss for batch: 0.728424072265625: 100%|██████████████████████████████████| 1130/1130 [26:55<00:00,  1.43s/it]


EPOCH 1 AVARAGE TRAINING LOSS: 1.5887181727232131

VALIDATION:


Val: 100%|███████████████████████████████████████████████████████████████████████████| 126/126 [00:57<00:00,  2.18it/s]


EM: 0.6032251642444754 | F1: 0.8239111647227957
Model saved!



Training loss for batch: 0.7442318797111511: 100%|█████████████████████████████████| 1130/1130 [28:16<00:00,  1.50s/it]


...

EM: 0.588094764085208 | F1: 0.8195551190479315



Training loss for batch: 0.265140563249588: 100%|██████████████████████████████████| 1130/1130 [26:56<00:00,  1.43s/it]


EPOCH 8 AVARAGE TRAINING LOSS: 0.23823581794524087

VALIDATION:


Val: 100%|███████████████████████████████████████████████████████████████████████████| 126/126 [00:57<00:00,  2.18it/s]

EM: 0.578339637666733 | F1: 0.8165371623308807



In [25]:
model.load_state_dict(torch.load('best_base_model.pt', weights_only=True))
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-3) 
epochs = 5

best_f1, best_exact_match = train(epochs, best_f1, best_exact_match)

Training loss for batch: 0.22260800004005432: 100%|████████████████████████████████| 1130/1130 [26:55<00:00,  1.43s/it]


EPOCH 1 AVARAGE TRAINING LOSS: 0.3744691622995697

VALIDATION:


Val: 100%|███████████████████████████████████████████████████████████████████████████| 126/126 [00:57<00:00,  2.17it/s]


EM: 0.6265180171212423 | F1: 0.8441167873761305
Model saved!



Training loss for batch: 0.43544548749923706: 100%|████████████████████████████████| 1130/1130 [26:56<00:00,  1.43s/it]


...

EM: 0.6215409118056938 | F1: 0.8408454656461882



Training loss for batch: 0.17843909561634064: 100%|████████████████████████████████| 1130/1130 [26:59<00:00,  1.43s/it]


EPOCH 5 AVARAGE TRAINING LOSS: 0.2101527809569266

VALIDATION:


Val: 100%|███████████████████████████████████████████████████████████████████████████| 126/126 [00:58<00:00,  2.17it/s]

EM: 0.6155683854270356 | F1: 0.8403517250590723



In [26]:
print(f'Получена модель с EM: {best_exact_match}, F1: {best_f1}')

Получена модель с EM: 0.6265180171212423, F1: 0.8441167873761305
